In [20]:
import numpy as np
import pandas as pd
path = u"./dataset/002.srt"
x = pd.read_csv(path, sep='\n')
y = np.array(x)

z = []
z.append(np.array(['1']))
for i in y:
    z.append(i)

for i in z:
    print(i)

s = ""
idx = 0
subPack = []
pack = []
for i in z:
    subPack.append(i[0])
    idx += 1
    if(idx % 3 == 0):
        pack.append(subPack)
        subPack = []
        idx = 0

start = []
end = []
for i in range(len(pack)):
    s = pack[i][1]
    s = s.replace(" --> ", "")
    s = s[0:8], s[12:20]
    a = list(s)
    pack[i][1] = a


['1']
['00:00:00,000 --> 00:00:12,809']
['核能發電到底是不是一個安全的技術呢？在核災之後，世界各國的核電發展又是如何呢？廢核是國際趨勢嗎？今天罩門透過重啟核四公投來聊聊核電究竟案板呈嗎？']
['2']
['00:00:12,809 --> 00:00:17,635']
['玩嗎']
['3']
['00:00:22,254 --> 00:01:21,894']
['哈囉大家好，我是自己換回到二二零二一公投，只撓著這個系列第四集呢？我們在前幾天已經談過公投，要不要綁大選藻礁公投，還有萊豬公投的三個議題，那如果你還沒看完歡迎你看完今天的影片之後回去看一看。這間這一集是第四集，我們要討論的是第十七案重啟核四公投，不過，我們發現這議題比想像中還要複雜很多，我們不需要先弄清楚許多關於核能還有盒子的資訊才有辦法進入後續的討論，所以這個主題呢？我們會分成上下兩集。今天這集我們會先來跟大家介紹核電廠揪竟然完全過去了？何在是怎麼回事也會聊到國際核的趨勢，以及台灣的核能發展現況，那明天我們就會正式進入這次核四公投的正反意見討論，當然照慣例，有在開來之前還是要先進了工商服務時間，台灣第一，場專門為約定影音創作者舉辦年會，一趟即將登場。嘍不然你是剛踏入影劇圈，還在努力殺出重圍佈置已經有點成果正在思考變現或其他問題上，']
['4']
['00:01:21,894 --> 00:02:04,210']
['入手這場年會都會幫助你突破困境，民眾知道下一步要怎麼走。這種年會由台灣新媒體影音創作者協會主辦，串聯了多位頂尖創作者，一起分享他們在內部策略，個人發展已經趨勢以及商業洞察的經驗的觀點。除此之外，呢？協會還邀請到月底官方代表科幻爲大家提供最準確的平台動向何建議呢？如果你還能創作者，但正在準備入行會是有興趣了解，當然也能參加，而是要免費的將在二零二零年的一月十二日登場，除了市長席位呢？也有線上影片票可以讓搭遠端參與，不僅優惠票種再來熱烈販售，當中呢？因為數量有限，所以有興趣者請點擊下方的雞卵連結了解更詳細的活動內容後，速速報名當']
['5']
['00:02:04,210 --> 00:02:36,297']
['就討論核四、我們必須要先瞭解一下核能電廠是怎麼運作的。真正發電廠的發電原理呢？是透過引發核分裂、連鎖反應釋放的龐大能量的燒水，再通過發電

In [21]:
pack[0]


['1',
 ['00:00:00', '00:00:12'],
 '核能發電到底是不是一個安全的技術呢？在核災之後，世界各國的核電發展又是如何呢？廢核是國際趨勢嗎？今天罩門透過重啟核四公投來聊聊核電究竟案板呈嗎？']

In [22]:
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array

config_path = "./model/bert_base/chinese_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/bert_base/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/bert_base/chinese_L-12_H-768_A-12/vocab.txt'


In [23]:
model = build_transformer_model(
    config_path=config_path, checkpoint_path=checkpoint_path, with_nsp=True
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)


In [24]:
token = []
seg = []
token_ids = ""
segment_ids = ""
result = []

In [25]:
# token_ids, segment_ids = tokenizer.encode(
#     pack[0][2], pack[1][2])
# token_ids, segment_ids = to_array([token_ids], [segment_ids])

# probas = model.predict([token_ids, segment_ids])[0]

In [26]:
# len(pack[3][2])
for i in pack:
    print (len(i[2]))

72
2
371
265
175
330
183
341
16
236
349
292
353
194
353
295
348
203


In [34]:
#pack[i][2]=pack[i][2][-1:-128]
print(pack[2][2][-128:])

灣的核能發展現況，那明天我們就會正式進入這次核四公投的正反意見討論，當然照慣例，有在開來之前還是要先進了工商服務時間，台灣第一，場專門為約定影音創作者舉辦年會，一趟即將登場。嘍不然你是剛踏入影劇圈，還在努力殺出重圍佈置已經有點成果正在思考變現或其他問題上，


In [35]:
for i in range(len(pack)-1):
    if (len(pack[i][2])+len(pack[i+1][2])>=512):
        pack[i][2]=pack[i][2][-128:]

    token_ids, segment_ids = tokenizer.encode(
        pack[i][2], pack[i+1][2])
    
    token_ids, segment_ids = to_array([token_ids], [segment_ids])

    probas = model.predict([token_ids, segment_ids])[0]
    print(probas)
    result.append(probas)


[0.88510215 0.11489788]
[0.11527702 0.884723  ]
[9.9999392e-01 6.0535062e-06]
[0.77045804 0.22954193]
[9.9999487e-01 5.1040524e-06]
[9.9999583e-01 4.1623553e-06]
[9.9999499e-01 5.0531353e-06]
[9.9960107e-01 3.9890152e-04]
[9.999341e-01 6.591202e-05]
[9.9999344e-01 6.5330000e-06]
[9.9999559e-01 4.3603964e-06]
[9.9999154e-01 8.4457151e-06]
[9.9999046e-01 9.4881680e-06]
[9.9998522e-01 1.4792718e-05]
[9.9999297e-01 7.0161718e-06]
[9.9999464e-01 5.3095805e-06]
[9.9999547e-01 4.4801591e-06]


In [36]:
pack

[['1',
  ['00:00:00', '00:00:12'],
  '核能發電到底是不是一個安全的技術呢？在核災之後，世界各國的核電發展又是如何呢？廢核是國際趨勢嗎？今天罩門透過重啟核四公投來聊聊核電究竟案板呈嗎？'],
 ['2', ['00:00:12', '00:00:17'], '玩嗎'],
 ['3',
  ['00:00:22', '00:01:21'],
  '灣的核能發展現況，那明天我們就會正式進入這次核四公投的正反意見討論，當然照慣例，有在開來之前還是要先進了工商服務時間，台灣第一，場專門為約定影音創作者舉辦年會，一趟即將登場。嘍不然你是剛踏入影劇圈，還在努力殺出重圍佈置已經有點成果正在思考變現或其他問題上，'],
 ['4',
  ['00:01:21', '00:02:04'],
  '入手這場年會都會幫助你突破困境，民眾知道下一步要怎麼走。這種年會由台灣新媒體影音創作者協會主辦，串聯了多位頂尖創作者，一起分享他們在內部策略，個人發展已經趨勢以及商業洞察的經驗的觀點。除此之外，呢？協會還邀請到月底官方代表科幻爲大家提供最準確的平台動向何建議呢？如果你還能創作者，但正在準備入行會是有興趣了解，當然也能參加，而是要免費的將在二零二零年的一月十二日登場，除了市長席位呢？也有線上影片票可以讓搭遠端參與，不僅優惠票種再來熱烈販售，當中呢？因為數量有限，所以有興趣者請點擊下方的雞卵連結了解更詳細的活動內容後，速速報名當'],
 ['5',
  ['00:02:04', '00:02:36'],
  '就討論核四、我們必須要先瞭解一下核能電廠是怎麼運作的。真正發電廠的發電原理呢？是透過引發核分裂、連鎖反應釋放的龐大能量的燒水，再通過發電機把這個能量轉化成電力，最後再透過冷卻水把多餘的廢物排掉。那目前世界上面多數的核電廠使用的燃料棒是濃度三到五二線等又二三五發電。而經過長時間的反應之後，呢？燃料棒真變成帶有放射性的核廢料，那這些核廢料該怎麼處裡呢？'],
 ['6',
  ['00:02:36', '00:03:35'],
  '價的需求決定是否運送到適合永久儲存、的最終處置場，但以高階核廢料來說，好了，會需要先泡水冷卻降溫再轉換為乾式貯存，那低階廢料的處理方式的中常會透過焚化、壓縮、固化等方式來處不處裡，最後再運送到暫存場地存放跟目前只有極少數國家能夠

In [38]:
token_ids


array([[ 101, 2518,  852, 1963, 3362, 2130, 1059, 2130, 2339, 8024, 5445,
         684, 1555, 6752, 4638, 6282, 8024, 1450, 8043,  809,  753, 7439,
         753, 7439, 7439, 2399, 4638, 3149, 3087,  889, 3683, 4212, 3425,
        1754, 2356,  753, 7439,  753, 7439, 2399,  758, 7953, 1724, 4636,
        1245, 6172, 5390, 2159, 7030,  679, 1168, 4680, 1184, 4158, 3632,
        8024, 1450, 8043, 3417, 1724, 6963, 3760, 3300, 6158, 1154, 1057,
        1378, 7442, 8024, 4638, 5543, 3975, 6211, 1205, 8024, 4534,  704,
        2772, 5442, 1062, 2832, 4634, 6629, 1184, 4638, 5520, 3250, 5548,
        5181, 8024, 2769,  947, 1044, 6341, 1168, 6857, 6174, 3300, 6929,
        7938, 8043, 3209, 1921, 2769,  947, 3298, 5262, 5265,  889, 4692,
        4692, 8024, 6857, 3613, 1062, 2832, 3633, 1353, 7427, 3175, 1146,
        1162, 2205, 2128, 1059, 2595, 1377, 6121, 2595,  102, 4994, 6917,
        3300, 3126, 4660,  676,  943, 7481,  784, 7938, 4692, 3791, 8043,
        6929, 4534, 4197, 8024, 1762, 

In [39]:
token_ids = np.array([[token_ids]])
segment_ids = np.array([[segment_ids]])
token_ids = token_ids.reshape(1,  1, -1)
segment_ids = segment_ids.reshape(1,  1, -1)


In [40]:
token_ids.shape, segment_ids.shape


((1, 1, 334), (1, 1, 334))

In [10]:
probas = model.predict([token_ids, segment_ids])[0]


In [41]:
for i in result:
    print(i[0])


0.88510215
0.11527702
0.9999939
0.77045804
0.9999949
0.9999958
0.999995
0.99960107
0.9999341
0.99999344
0.9999956
0.99999154
0.99999046
0.9999852
0.99999297
0.99999464
0.99999547


In [12]:
# token_ids, segment_ids = tokenizer.encode(
#     pack[0][2],
#     pack[1][2])
# token_ids, segment_ids = to_array([token_ids], [segment_ids])
# token_ids.shape
# probas = model.predict([token_ids, segment_ids])[0]
# probas


In [13]:
pack


[['1',
  ['00:00:00', '00:00:59'],
  '在就一心有一個報導提出，目前針對早期新冠肺炎的唯一治療方法，那很多一聲卻不知道，所以想請問您不是這篇報導所說的，唯一治療方法是哪一種治療方法呢？今天的報導，這個也搞的有點神色醫師喔？現在看那個標題寫的好像有一個靈丹妙藥，大家都給忽視掉，那其實並不是這樣的。那篇報導裡面提到一九九四單克隆抗體療法，那實際上，美帝陸續有報導，那麼他這個改版都只是要強調就是很多醫生，其實並沒有很充分的陰影，有這樣一個工具就是當克隆抗體的這個治療的方案來，真的一些病人特別是對於早期患有無精症，阿或者是中度症狀，的病人，還有一些情況可以用來去向來是怕被來這就是預防性的治療，所以這份報告，我覺得就是提醒而已。省，其實你是有一個額外的工具是可以用的，並不是說，病人來了。'],
 ['1',
  ['00:00:59', '00:01:29'],
  '確診以後，女高才真的回家看的，但如果病情有沒有惡化，那時候你的拿鐵，我們這邊受不了，什麼就是不要有這樣。一個一個臺獨還是知道有一些工具的。當然其實除了這個，但可能看你都還是有一些其他的藥物，包括一些小人，第四個呀，或者是其他的一些喔。現在比如說這個阿曼天啊等下，要我對於早期治療其實也是有作用的，所以就是我覺得是醫生要有更多的這個知識儲備，'],
 ['2',
  ['00:01:29', '00:01:42'],
  '那這方面，我們就可以稍微具體看一下，因為這個當可能看你的療法技術。去年開始就已經有這個抗體。獲得的第一的這個意味就是緊急使用。許可'],
 ['3',
  ['00:01:42', '00:01:54'],
  '那這些獲得許可的當口看聽就要比如說是煎熬喔。這個公式的這個阿丹可能看起來真的，那麼這裡面呢？就牽扯到。就是說，'],
 ['4',
  ['00:01:54', '00:02:53'],
  '跟著提到的很大一部分的這個使用的用途呢？它是用來預防的，就你跟比如說，做這個皮下檢測發現自己被干擾了？那麼你還沒有嚴重的徵狀出現也沒有甚至住院那麼這種情況下呢？你用繼續當可能看你的治療的話，效果相對來說是比較明顯的，就是是根據臨床試驗來看的，那這裡面有提到，其實下麵可以買或者是影帝，也沒那麼，基本上，這個呢？就是有六百毫克，兩個都有六百毫克，這個就是可以起到一

In [14]:
for i in range(len(pack)-1):
    pack[i].append(result[i][0])


In [15]:
pack


[['1',
  ['00:00:00', '00:00:59'],
  '在就一心有一個報導提出，目前針對早期新冠肺炎的唯一治療方法，那很多一聲卻不知道，所以想請問您不是這篇報導所說的，唯一治療方法是哪一種治療方法呢？今天的報導，這個也搞的有點神色醫師喔？現在看那個標題寫的好像有一個靈丹妙藥，大家都給忽視掉，那其實並不是這樣的。那篇報導裡面提到一九九四單克隆抗體療法，那實際上，美帝陸續有報導，那麼他這個改版都只是要強調就是很多醫生，其實並沒有很充分的陰影，有這樣一個工具就是當克隆抗體的這個治療的方案來，真的一些病人特別是對於早期患有無精症，阿或者是中度症狀，的病人，還有一些情況可以用來去向來是怕被來這就是預防性的治療，所以這份報告，我覺得就是提醒而已。省，其實你是有一個額外的工具是可以用的，並不是說，病人來了。',
  0.99999475],
 ['1',
  ['00:00:59', '00:01:29'],
  '確診以後，女高才真的回家看的，但如果病情有沒有惡化，那時候你的拿鐵，我們這邊受不了，什麼就是不要有這樣。一個一個臺獨還是知道有一些工具的。當然其實除了這個，但可能看你都還是有一些其他的藥物，包括一些小人，第四個呀，或者是其他的一些喔。現在比如說這個阿曼天啊等下，要我對於早期治療其實也是有作用的，所以就是我覺得是醫生要有更多的這個知識儲備，',
  0.9999844],
 ['2',
  ['00:01:29', '00:01:42'],
  '那這方面，我們就可以稍微具體看一下，因為這個當可能看你的療法技術。去年開始就已經有這個抗體。獲得的第一的這個意味就是緊急使用。許可',
  0.99999404],
 ['3',
  ['00:01:42', '00:01:54'],
  '那這些獲得許可的當口看聽就要比如說是煎熬喔。這個公式的這個阿丹可能看起來真的，那麼這裡面呢？就牽扯到。就是說，',
  0.9999937],
 ['4',
  ['00:01:54', '00:02:53'],
  '跟著提到的很大一部分的這個使用的用途呢？它是用來預防的，就你跟比如說，做這個皮下檢測發現自己被干擾了？那麼你還沒有嚴重的徵狀出現也沒有甚至住院那麼這種情況下呢？你用繼續當可能看你的治療的話，效果相對來說是比較明顯的，就是是根據臨床試驗來看的，那這裡面有提

In [16]:

idxList = []
lessThanHalf = []
lessThanHalf.append('00:00:00')
for i in range(1, len(pack)-1):
    if(pack[i][3] < 0.2):
        idxList.append(i)
        lessThanHalf.append(pack[i][1][1])


In [17]:
combineText = []

S = pack[0][2]
for i in range(len(pack)-1):

    token_testId, Seg_testId = tokenizer.encode(pack[i][2], pack[i+1][2])
    token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
    result = model.predict([token_testId, Seg_testId])[0][0]
    # print(pack[i][2])
    if(result > 0.98):
        S += pack[i+1][2]

    else:
        S += pack[i+1][2]
        combineText.append(S)
        S = ""


In [18]:
combineText


['在就一心有一個報導提出，目前針對早期新冠肺炎的唯一治療方法，那很多一聲卻不知道，所以想請問您不是這篇報導所說的，唯一治療方法是哪一種治療方法呢？今天的報導，這個也搞的有點神色醫師喔？現在看那個標題寫的好像有一個靈丹妙藥，大家都給忽視掉，那其實並不是這樣的。那篇報導裡面提到一九九四單克隆抗體療法，那實際上，美帝陸續有報導，那麼他這個改版都只是要強調就是很多醫生，其實並沒有很充分的陰影，有這樣一個工具就是當克隆抗體的這個治療的方案來，真的一些病人特別是對於早期患有無精症，阿或者是中度症狀，的病人，還有一些情況可以用來去向來是怕被來這就是預防性的治療，所以這份報告，我覺得就是提醒而已。省，其實你是有一個額外的工具是可以用的，並不是說，病人來了。確診以後，女高才真的回家看的，但如果病情有沒有惡化，那時候你的拿鐵，我們這邊受不了，什麼就是不要有這樣。一個一個臺獨還是知道有一些工具的。當然其實除了這個，但可能看你都還是有一些其他的藥物，包括一些小人，第四個呀，或者是其他的一些喔。現在比如說這個阿曼天啊等下，要我對於早期治療其實也是有作用的，所以就是我覺得是醫生要有更多的這個知識儲備，那這方面，我們就可以稍微具體看一下，因為這個當可能看你的療法技術。去年開始就已經有這個抗體。獲得的第一的這個意味就是緊急使用。許可那這些獲得許可的當口看聽就要比如說是煎熬喔。這個公式的這個阿丹可能看起來真的，那麼這裡面呢？就牽扯到。就是說，跟著提到的很大一部分的這個使用的用途呢？它是用來預防的，就你跟比如說，做這個皮下檢測發現自己被干擾了？那麼你還沒有嚴重的徵狀出現也沒有甚至住院那麼這種情況下呢？你用繼續當可能看你的治療的話，效果相對來說是比較明顯的，就是是根據臨床試驗來看的，那這裡面有提到，其實下麵可以買或者是影帝，也沒那麼，基本上，這個呢？就是有六百毫克，兩個都有六百毫克，這個就是可以起到一定的預防的作用，那這個其實也比較有意思，這個應該是當肯抗體的雞尾酒療法，的，嗎？那麼這兩款都是你覺得公司開發出來的獲得的。許可，那其實他們最早的用量是一千兩百毫克很難後來給了降下來了。到六百毫克，覺得就是有一些臨床先支持說，並不需要那麼高的用途上，其實也做了一些就是怎麼樣使用，這方面也做了一點調整，就原來的話，只有就要強調全部都是靜脈注射阿姨注射，那麼現在也許有一些皮下注射，但是皮下就是比較麻煩，就是你要

In [19]:
dataString = []
for i in range(len(idxList)):
    dataString.append(pack[i][2])


In [20]:
Str = []
a = ''
b = 0
k = 0
for i in range(len(idxList)):
    if(i == idxList[k]):
        for j in range(b, i+1):
            a += pack[j][2]
        Str.append(a)
        k += 1
        a = ''
        b = i+1


In [21]:
# Summary.ipynb
from __future__ import print_function
import numpy as np
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
from keras.models import Model
from rouge import Rouge  # pip install rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [22]:
maxlen = 256
batch_size = 16
epochs = 20

config_path = "./model/bert_base/chinese_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/bert_base/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/bert_base/chinese_L-12_H-768_A-12/vocab.txt'

'''
from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForCausalLM,
   AutoModelForTokenClassification,
)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForMaskedLM.from_pretrained('ckiplab/albert-tiny-chinese') # or other models above
'''


"\nfrom transformers import (\n   BertTokenizerFast,\n   AutoModelForMaskedLM,\n   AutoModelForCausalLM,\n   AutoModelForTokenClassification,\n)\n\ntokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')\nmodel = AutoModelForMaskedLM.from_pretrained('ckiplab/albert-tiny-chinese') # or other models above\n"

In [23]:
def load_data(filename):
    D = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            title, content = l.strip().split('\t')
            D.append((title, content))
    return D


In [24]:
train_data = load_data('./dataset/train.tsv')
'''
valid_data = load_data('/root/csl/val.tsv')
test_data = load_data('/root/csl/test.tsv')
'''


"\nvalid_data = load_data('/root/csl/val.tsv')\ntest_data = load_data('/root/csl/test.tsv')\n"

In [25]:
token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)


In [26]:
class CrossEntropy(Loss):
    def compute_loss(self, inputs, mask=None):
        y_true, y_mask, y_pred = inputs
        y_true = y_true[:, 1:]
        y_mask = y_mask[:, 1:]
        y_pred = y_pred[:, :-1]
        loss = K.sparse_categorical_crossentropy(y_true, y_pred)
        loss = K.sum(loss * y_mask) / K.sum(y_mask)
        return loss


In [27]:
model = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,
)

output = CrossEntropy(2)(model.inputs + model.outputs)

model = Model(model.inputs, output)


In [28]:
class AutoTitle(AutoRegressiveDecoder):
    @AutoRegressiveDecoder.wraps(default_rtype='probas')
    def predict(self, inputs, output_ids, states):
        token_ids, segment_ids = inputs
        token_ids = np.concatenate([token_ids, output_ids], 1)
        segment_ids = np.concatenate(
            [segment_ids, np.ones_like(output_ids)], 1)
        return self.last_token(model).predict([token_ids, segment_ids])

    def generate(self, text, topk=1):
        max_c_len = maxlen - self.maxlen
        token_ids, segment_ids = tokenizer.encode(text, maxlen=max_c_len)
        output_ids = self.beam_search([token_ids, segment_ids],
                                      topk=topk)
        return tokenizer.decode(output_ids)


autotitle = AutoTitle(start_id=None, end_id=tokenizer._token_end_id, maxlen=32)


In [29]:
model.load_weights('./best_model/best_model.weights')


In [30]:
StrAuto = []
for i in combineText:
    s = autotitle.generate(i)
    StrAuto.append(s)


In [31]:
StrAuto


['面向早期新冠肺炎的克隆抗體治療方法研究']